In [1]:
from glob import glob
import librosa
import soundfile as sf
import numpy as np
import pandas as pd
import sys
import time
import datetime
from tqdm import tqdm
import random
import os
import gc
import cv2

from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, AdamW, lr_scheduler
from torch.distributions import Uniform
from torch.utils.data import DataLoader, Dataset

from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation

from efficientnet_pytorch import EfficientNet
from torchvision.models import resnet34, resnet50

import sys
sys.path.append('..')
from libs import transform as tr
from libs import spectrogram as spec
from libs import criterion as cr

import warnings
warnings.filterwarnings("ignore")

/home/yuigahama/anaconda3/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [2]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = False  # type: ignore
set_seed(53)

In [3]:
train_tp = pd.read_csv('../../data/train_tp.csv')
train_fp = pd.read_csv('../../data/train_fp.csv')
submission = pd.read_csv('../../data/sample_submission.csv')

pred_target = list(submission.columns)[1:]

SR = 48000

In [4]:
def normalize_melspec(X: np.ndarray):
    eps = 1e-6
    mean = X.mean()
    X = X - mean
    std = X.std()
    Xstd = X / (std + eps)
    norm_min, norm_max = Xstd.min(), Xstd.max()
    if (norm_max - norm_min) > eps:
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

def save(fold, model, optim, criterion, file_path="../../model/"):
    if not TEST_NAME in os.listdir(file_path):
        os.mkdir(file_path+TEST_NAME)
    
    
    output_path = file_path + TEST_NAME + '/' + f"{TEST_NAME}_{fold}.model"
    
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.cpu().state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'criterion': criterion
        },
        output_path)
    
    model.to(device)
    
    return output_path

In [5]:
class RfcxDataSet(Dataset):
    def __init__(self,
                 tp:pd.DataFrame,
                 train: bool,
                 data_path:str,
                 pcen_parameters:dict,
                 pre_calc=True,
                 n_mels=128
    ):
        self.tp = tp
        self.path = data_path
        self.img_size = 256
        self.train = train
        self.n_mels = n_mels
        self.pre_calc = pre_calc
        
        self.transform = tr.Compose([
            tr.OneOf([
                tr.GaussianNoiseSNR(min_snr=10),
                tr.PinkNoiseSNR(min_snr=10)
            ]),
            tr.PitchShift(max_steps=2, sr=SR),
            #tr.TimeStretch(),
            #tr.TimeShift(sr=sr),
            tr.VolumeControl(mode="sine")
        ])
        
        self.pcen_parameters = pcen_parameters
        
    def __len__(self):
        return len(self.tp)
    
    def load(self, record_path):
        y, orig_sr = sf.read(record_path)
        
        if orig_sr != SR:
            y = librosa.resample(y, orig_sr=orig_sr, target_sr=SR, res_type="kaiser_best")
        return y
    
    def get_random_duration(self, duration=10):
        start_sec = random.randint(0, 60-duration)
        end_sec = start_sec + 10
            
        return start_sec, end_sec
    
    def get_duration(self, t_min, t_max, duration=10):
        annotated_duration = t_max - t_min
        
        if annotated_duration > duration:
            limit_sec = t_max - duration
            start_sec = random.randint(t_min, limit_sec)
            end_sec = start_sec + duration

        else:
            res_time = duration - annotated_duration
            front_limit = res_time if res_time < t_min else t_min
            
            front_time = random.randint(0, front_limit)
            
            back_limit = 60 - t_max
            
            tmp_time = res_time - front_time
            back_time = tmp_time if tmp_time < back_limit else back_limit
            
            if not tmp_time < back_limit:
                front_time += tmp_time - back_limit
            
            start_sec = t_min - front_time
            end_sec = t_max + back_time
            
        return start_sec, end_sec
    
    def create_mel(self, y):
        y = self.transform(y)
        
        melspec = librosa.feature.melspectrogram(
            y,
            sr=SR,
            fmin=0,
            fmax=15000,
            n_mels=128
        )

        pcen = librosa.pcen(melspec, sr=SR, **self.pcen_parameters)
        clean_mel = librosa.power_to_db(melspec ** 1.5)
        melspec = librosa.power_to_db(melspec)

        norm_melspec = normalize_melspec(melspec)
        norm_pcen = normalize_melspec(pcen)
        norm_clean_mel = normalize_melspec(clean_mel)

        image = np.stack([norm_melspec, norm_pcen, norm_clean_mel], axis=-1)

        return image
    
    def __getitem__(self, idx: int):
        sample = self.tp.iloc[idx, :]
        recording_id = sample['recording_id']
        t_min = int(round(sample['t_min']))
        t_max = int(round(sample['t_max']))
        
        start_sec, end_sec = self.get_duration(t_min, t_max, 10)
            
        record_path = self.path + recording_id + '.flac'
        y = self.load(record_path)
        y =  y[start_sec*SR:end_sec*SR]
        
        if self.train:
            y = self.transform(y)
        
        species_id = sample['species_id']
        target = torch.zeros([24], dtype=torch.float32)
        target[species_id] = 1
        
        return y, target

In [6]:
def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)


def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)


def init_weights(model):
    classname = model.__class__.__name__
    if classname.find("Conv2d") != -1:
        nn.init.xavier_uniform_(model.weight, gain=np.sqrt(2))
        model.bias.data.fill_(0)
    elif classname.find("BatchNorm") != -1:
        model.weight.data.normal_(1.0, 0.02)
        model.bias.data.fill_(0)
    elif classname.find("GRU") != -1:
        for weight in model.parameters():
            if len(weight.size()) > 1:
                nn.init.orghogonal_(weight.data)
    elif classname.find("Linear") != -1:
        model.weight.data.normal_(0, 0.01)
        model.bias.data.zero_()


def do_mixup(x: torch.Tensor, mixup_lambda: torch.Tensor):
    """Mixup x of even indexes (0, 2, 4, ...) with x of odd indexes
    (1, 3, 5, ...).
    Args:
      x: (batch_size * 2, ...)
      mixup_lambda: (batch_size * 2,)
    Returns:
      out: (batch_size, ...)
    """
    out = (x[0::2].transpose(0, -1) * mixup_lambda[0::2] +
           x[1::2].transpose(0, -1) * mixup_lambda[1::2]).transpose(0, -1)
    return out


class Mixup(object):
    def __init__(self, mixup_alpha, random_seed=1234):
        """Mixup coefficient generator.
        """
        self.mixup_alpha = mixup_alpha
        self.random_state = np.random.RandomState(random_seed)

    def get_lambda(self, batch_size):
        """Get mixup random coefficients.
        Args:
          batch_size: int
        Returns:
          mixup_lambdas: (batch_size,)
        """
        mixup_lambdas = []
        for n in range(0, batch_size, 2):
            lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
            mixup_lambdas.append(lam)
            mixup_lambdas.append(1. - lam)

        return torch.from_numpy(np.array(mixup_lambdas, dtype=np.float32))


def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.
    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1:, :].repeat(
        1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output

In [7]:
class AttBlock(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear",
                 temperature=1.0):
        super().__init__()

        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.bn_att = nn.BatchNorm1d(out_features)
        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)
        init_bn(self.bn_att)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.clamp(self.att(x), -10, 10), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)


class AttBlockV2(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear"):
        super().__init__()

        self.activation = activation
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)

In [8]:
class EfficientNetSED(nn.Module):
    def __init__(
        self,
        base_model_name: str,
        pretrained=False,
        num_classes=24,
        spectrogram_params={},
        logmel_extractor_params={},
        spec_augmenter_params={},
        pce_params={}
    ):
        super().__init__()
        self.spectrogram_extractor = Spectrogram(**spectrogram_params)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(**logmel_extractor_params)
        
        #Pcen converter
        self.pcen_converter = spec.pcen(**pce_params)

        # Spec augmenter
        #self.spec_augmenter = SpecAugmentation(**spec_augmenter_params)
        
        self.interpolate_ratio = 30  # Downsampled ratio
        self.mixup_alpha = 0.2
        self.random_state = np.random.RandomState(123)
        
        if pretrained:
            self.base_model = EfficientNet.from_pretrained(base_model_name)
        else:
            self.base_model = EfficientNet.from_name(base_model_name)

        in_features = self.base_model._fc.in_features

        self.fc1 = nn.Linear(in_features, in_features, bias=True)
        self.att_block = AttBlockV2(in_features, num_classes, activation="sigmoid")

        self.init_weight()
        
    def mixup(self, x):
        sizws = x.size()
        #lam = torch.from_numpy(self.random_state.beta(self.mixup_alpha, self.mixup_alpha, (sizws[0], 1))).cuda()
        lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
        index = list(range(x.size(0)))
        random.shuffle(index)
        #out = (x.view(sizws[0], -1) * lam + x[index].squeeze().view(sizws[0], -1) * (1-lam)).view(sizws[0], sizws[1], sizws[2], sizws[3],)
        out = (x * lam + x[index].squeeze() * (1-lam))
        return out.float(), {'lam': lam, 'index': index}

    def init_weight(self):
        init_layer(self.fc1)

    def forward(self, input):        
        x = self.spectrogram_extractor(input)
        x = self.logmel_extractor(x)
        
        x_mels = self.logmel_extractor.power_to_db(x)
        x_pcen = self.pcen_converter(x) 
        x_clear = self.logmel_extractor.power_to_db(x ** 1.5)
        
        
        x = torch.cat((x_mels,x_pcen,x_clear),1)
        #x = torch.cat((x,x,x),1)
        
        frames_num = x.size(2)
        
        if self.training:
            x, mix_info = self.mixup(x)
            #x = self.spec_augmenter(x)
        else:
            mix_info = None
                
        # (batch_size, channels, freq, frames)
        x = self.base_model.extract_features(x)

        # (batch_size, channels, frames)
        x = torch.mean(x, dim=3)

        # channel smoothing
        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)
        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_logit = self.att_block.cla(x).transpose(1, 2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output,
                                       self.interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        framewise_logit = interpolate(segmentwise_logit, self.interpolate_ratio)
        framewise_logit = pad_framewise_output(framewise_logit, frames_num)

        output_dict = {
            "framewise_output": framewise_output,
            "segmentwise_output": segmentwise_output,
            "logit": logit,
            "framewise_logit": framewise_logit,
            "clipwise_output": clipwise_output
        }

        return output_dict, mix_info

In [9]:
# LRAP. Instance-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = (scores.sum(-1) / labels.sum(-1)).mean()
    return score.item()

# label-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LWLRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    # Number of GT labels per instance
    num_labels = labels.sum(-1)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = scores.sum() / labels.sum()
    return score.item()

def mixup_socre(cor, x, y, mix_info):
    return cor(x, y) * mix_info['lam'] + cor(x, y[mix_info['index']].squeeze()) * (1-mix_info['lam'])

In [10]:
TEST_NAME = 'efficient-b0-best'

n_splits = 5
random_state = 1
epochs = 35

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

spectrogram_params = {
    'n_fft': 2048,
    'hop_length': 512,
    'win_length': 2048,
    'window': 'hann',
    'center': True,
    'pad_mode': 'reflect',
    'freeze_parameters': True
}

logmel_extractor_params = {
    'sr': SR,
    'n_fft': 2048,
    'n_mels': 384,
    'fmin': 0,
    'fmax': 15000,
    'ref' : 1.0,
    'amin': 1e-10,
    'top_db': None,
    'is_log': False,
    'freeze_parameters': True
}

spec_augmenter_params = {
    'time_drop_width':  64,
    'time_stripes_num': 2,
    'freq_drop_width':  8,
    'freq_stripes_num': 2
}

pce_params = {
    'gain': 0.98,
    'bias': 2,
    'power': 0.5,
    'time_constant': 0.4,
    'eps': 0.000001,
}

model_params = {
    'base_model_name': 'efficientnet-b0',
    'pretrained': True,
    'num_classes': 24,
    'spectrogram_params': spectrogram_params,
    'logmel_extractor_params': logmel_extractor_params,
    'spec_augmenter_params': spec_augmenter_params,
    'pce_params': pce_params
}


optim_params = {
    'lr': 1e-3,
    'weight_decay': 5e-5,
    'betas': (0.9, 0.999)
}

scheduler_params = {
    'mode': 'max',
    'patience': 1,
    'factor': 0.6,
    'verbose': False
}

pcen_parameters = {
    'gain': 0.98,
    'bias': 2,
    'power': 0.5,
    'time_constant': 0.4,
    'eps': 0.000001,
}

train_params = {
    'pcen_parameters': pcen_parameters,
    'pre_calc': False,
    'train': True,
    'data_path': '/home/yuigahama/kaggle/rfcx/data/train/'  
}

val_params = {
    'train': True,
    'pre_calc': True,
    'pcen_parameters': pcen_parameters,
    'data_path': '/home/yuigahama/kaggle/rfcx/data/train/'
}

test_data_params = {
    'train': False,
    'path': '/home/yuigahama/kaggle/rfcx/data/test_wo_fp/'
}

dataloder_params = {
    'batch_size': 16,
    'num_workers': 15,
    'pin_memory': False,
}

test_dataloder_params = {
    'batch_size': 32,
    'num_workers': 15,
    'pin_memory': False,
    'shuffle':False
}

In [ ]:
tta = np.zeros((len(submission), 24))
cv_score = 0
lrs = np.arange(5e-4,1.0e-3,5e-5).tolist()

for fold_id, (train_index, val_index) in enumerate(skf.split(train_tp, train_tp.species_id)):
    print(f'---------- fold {fold_id} ----------')
    
    model = EfficientNetSED(**model_params).to(device)
    optim = Adam(model.parameters(), **optim_params)
    #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer=optim, **scheduler_params)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer=optim, T_max=epochs-10)
    
    pos_weights = torch.ones(24)
    pos_weights = pos_weights * 24
    criterion = cr.ImprovedFocalLoss(weights=[1, 0.5])
    
    train_dataset = RfcxDataSet(train_tp.iloc[train_index], **train_params)
    val_dataset   = RfcxDataSet(train_tp.iloc[val_index], **val_params)

    train_dataloader = DataLoader(train_dataset, shuffle=True, **dataloder_params)
    val_dataloader = DataLoader(val_dataset, shuffle=False, **dataloder_params)
    
    es = 50
    bast_score = 0
    for epoch in range(1, epochs):
        if es <= 0:
            break
        
        
        if epoch <= 10:
            for g in optim.param_groups:
                g['lr'] = lrs[epoch-1]
        else:
            scheduler.step(val_score)
        
        
        start_time = time.time()
        
        # train
        model.train()
        train_loss = 0
        train_score = 0
        train_corr = 0
        
        for data in train_dataloader:
            image = data[0].float().to(device)
            label = data[1]
            
            optim.zero_grad()
            output, mix_info = model(image)
            
            output = {k:v.cpu() for k,v in output.items()}
            
            loss = mixup_socre(criterion, output, label, mix_info)
            #loss = criterion(output, label, mix_info)
            pred_labels = output["framewise_output"].max(1)[0]
            score = mixup_socre(LWLRAP, pred_labels, label, mix_info)
            
            #score = LWLRAP(pred_labels, label)
                        
            loss.backward()
            optim.step()
            
            vals, answers = torch.max(pred_labels, 1)
            vals, targets = torch.max(label, 1)
            vals, targets2 = torch.max(label[mix_info['index']], 1)
            
            corrects = 0
            for i in range(0, len(answers)):
                if answers[i] == targets[i]:
                    corrects = corrects + 1
                if answers[i] == targets2[i]:
                    corrects = corrects + 1
                
                corrects = 1 if corrects > 0 else 0
                    
            
            train_corr += corrects
            train_loss += loss.item()
            train_score += score
            
        train_loss  /= len(train_dataloader)
        train_score /= len(train_dataloader)
        
        # val
        model.eval()
        val_loss = 0
        val_score = 0
        val_corr = 0

        with torch.no_grad():
            for val_data in val_dataloader:
                image = val_data[0].float().to(device)
                label = val_data[1]

                output, mix_info = model(image)
                output = {k:v.cpu() for k,v in output.items()}
                
                pred_labels = output["framewise_output"].max(1)[0]
                vals, answers = torch.max(pred_labels, 1)
                vals, targets = torch.max(label, 1)
                
                corrects = 0
                for i in range(0, len(answers)):
                    if answers[i] == targets[i]:
                        corrects = corrects + 1

                val_corr += corrects
                val_loss += criterion(output, label)
                val_score += LWLRAP(pred_labels, label)
                
        val_loss  /= len(val_dataloader)
        val_score /= len(val_dataloader)
        
        duration = str(datetime.timedelta(seconds=time.time() - start_time))[:7]
        print(f'E {epoch:3}| T | L: {train_loss:.3} | S: {train_score:.3} | C: {train_corr}/{len(train_dataset)} | V | L: {val_loss:.3} | S: {val_score:.3} | C: {val_corr}/{len(val_dataset)} | T: {duration} | es: {es}')

        if bast_score < val_score:
            bast_score = val_score
            bast_path = save(fold_id, model, optim, criterion)
        else:
            es -= 1
        
        if es <= 0:
            break
            
        #scheduler.step(val_score)
    
    print(f"bast score: {bast_score}")
    del model, train_dataset, val_dataset, train_dataloader, val_dataloader, optim
    gc.collect()

---------- fold 0 ----------
Loaded pretrained weights for efficientnet-b0
E   1| T | L: 3.88 | S: 0.209 | C: 48/972 | V | L: 3.09 | S: 0.171 | C: 13/244 | T: 0:00:36 | es: 50
E   2| T | L: 1.92 | S: 0.292 | C: 59/972 | V | L: 2.65 | S: 0.26 | C: 25/244 | T: 0:00:37 | es: 50
E   3| T | L: 1.72 | S: 0.381 | C: 61/972 | V | L: 2.16 | S: 0.396 | C: 60/244 | T: 0:00:36 | es: 50
E   4| T | L: 1.57 | S: 0.482 | C: 61/972 | V | L: 1.48 | S: 0.494 | C: 75/244 | T: 0:00:37 | es: 50
E   5| T | L: 1.5 | S: 0.517 | C: 61/972 | V | L: 1.33 | S: 0.538 | C: 87/244 | T: 0:00:36 | es: 50
E   6| T | L: 1.36 | S: 0.588 | C: 61/972 | V | L: 1.09 | S: 0.631 | C: 111/244 | T: 0:00:36 | es: 50
E   7| T | L: 1.25 | S: 0.644 | C: 61/972 | V | L: 1.05 | S: 0.698 | C: 133/244 | T: 0:00:36 | es: 50
E   8| T | L: 1.15 | S: 0.706 | C: 61/972 | V | L: 1.06 | S: 0.689 | C: 127/244 | T: 0:00:36 | es: 50
E   9| T | L: 1.12 | S: 0.707 | C: 61/972 | V | L: 0.97 | S: 0.737 | C: 151/244 | T: 0:00:36 | es: 49
E  10| T | L: 

In [11]:
def create_mel(y, img_size=256): 
    melspec = librosa.feature.melspectrogram(
        y,
        sr=SR,
        fmin=0,
        fmax=15000,
        n_mels=128
    )

    pcen = librosa.pcen(melspec, sr=SR, **pcen_parameters)
    clean_mel = librosa.power_to_db(melspec ** 1.5)
    melspec = librosa.power_to_db(melspec)

    norm_melspec = normalize_melspec(melspec)
    norm_pcen = normalize_melspec(pcen)
    norm_clean_mel = normalize_melspec(clean_mel)

    image = np.stack([norm_melspec, norm_pcen, norm_clean_mel], axis=-1)
    
    height, width, _ = image.shape
    image = cv2.resize(image, (img_size * 2, img_size))
    image = np.moveaxis(image, 2, 0)
    image = (image / 255.0).astype(np.float32)

    return image

def prediction_for_clip(audio_id: str,
                        clip: np.ndarray, 
                        model: EfficientNetSED,
                        threshold=0.5):
    PERIOD = 10
    audios = []
    y = clip.astype(np.float32)
    len_y = len(y)
    start = 0
    end = PERIOD * SR
    while True:
        y_batch = y[start:end].astype(np.float32)

        start = end - (5 * SR)
        end += 5 * SR
        
        #mel = create_mel(y_batch)
        audios.append(y_batch)
        
        if len_y < end:
            break
            
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    array = np.asarray(audios)
    image = torch.from_numpy(array).to(device)
    
    model.eval()
    estimated_event_list = []
    global_time = 0.0
    

    with torch.no_grad():
        prediction, _ = model(image)
        frame_pred = torch.sum(
            torch.sigmoid(torch.max(prediction["framewise_output"], 1)[0]), 0
        ).detach().cpu().numpy()
        framewise_outputs = torch.max(prediction["framewise_output"], 0)[0].detach(
            ).cpu().numpy()
        
    thresholded = framewise_outputs >= threshold
    
    for target_idx in range(thresholded.shape[1]):
        if thresholded[:, target_idx].mean() == 0:
            pass
        else:
            detected = np.argwhere(thresholded[:, target_idx]).reshape(-1)
            head_idx = 0
            tail_idx = 0
            while True:
                if (tail_idx + 1 == len(detected)) or (
                        detected[tail_idx + 1] - 
                        detected[tail_idx] != 1):
                    onset = 0.01 * detected[
                        head_idx] + global_time
                    offset = 0.01 * detected[
                        tail_idx] + global_time
                    onset_idx = detected[head_idx]
                    offset_idx = detected[tail_idx]
                    max_confidence = framewise_outputs[
                        onset_idx:offset_idx, target_idx].max()
                    mean_confidence = framewise_outputs[
                        onset_idx:offset_idx, target_idx].mean()
                    estimated_event = {
                        "audio_id": audio_id,
                        "ebird_code": target_idx,
                        "onset": onset,
                        "offset": offset,
                        "max_confidence": max_confidence,
                        "mean_confidence": mean_confidence
                    }
                    estimated_event_list.append(estimated_event)
                    head_idx = tail_idx + 1
                    tail_idx = tail_idx + 1
                    if head_idx >= len(detected):
                        break
                else:
                    tail_idx += 1
        global_time += PERIOD
        
    prediction_df = pd.DataFrame(estimated_event_list)
    return prediction_df, frame_pred

In [12]:
def prediction(test_df: pd.DataFrame,
               model: dict,
               threshold=0.5):
    #model = get_model(model_config, weights_path)
    unique_audio_id = test_df.recording_id.unique()

    warnings.filterwarnings("ignore")
    prediction_dfs = []
    frame_dict = dict()
    for audio_id in tqdm(unique_audio_id):
        clip, _ = librosa.load(
            f'/home/yuigahama/kaggle/rfcx/data/test/{audio_id}.flac',
            sr=SR,
            mono=True,
            res_type="kaiser_fast"
        )
        
        test_df_for_audio_id = test_df.query(
            f"recording_id == '{audio_id}'").reset_index(drop=True)
        prediction_df, frame_pred = prediction_for_clip(
            audio_id,
            clip=clip,
            model=model,
            threshold=threshold
        )
        frame_dict[audio_id] = frame_pred
        prediction_dfs.append(prediction_df)
    
    prediction_df = pd.concat(prediction_dfs, axis=0, sort=False).reset_index(drop=True)
    return prediction_df, frame_dict

In [13]:
preds = []
frams = [] 
for i in range(5):

    model = EfficientNetSED(**model_params).to(device)
    params = torch.load(f'/home/yuigahama/kaggle/rfcx/model/{TEST_NAME}/{TEST_NAME}_{i}.model')
    model.load_state_dict(params['model_state_dict'])
    
    prediction_df, frame_dict = prediction(
        test_df=submission,
        model=model,
        threshold=0.5
    )
    print(len(prediction_df.audio_id.unique()))
    preds.append(prediction_df)
    frams.append(pd.DataFrame(frame_dict).T)

  0%|          | 0/1992 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b0


100%|██████████| 1992/1992 [03:01<00:00, 10.97it/s]


1943


  0%|          | 0/1992 [00:00<?, ?it/s]

...

1901


  0%|          | 0/1992 [00:00<?, ?it/s]

Loaded pretrained weights for efficientnet-b0


100%|██████████| 1992/1992 [03:01<00:00, 10.96it/s]


1946


In [15]:
sub = pd.DataFrame(np.zeros((len(submission), 24)),columns=pred_target, index=submission['recording_id'])
for p,j in zip(frams, [0.1, 0.225, 0.225, 0.225, 0.225]):
    p.columns = pred_target
    sub += (p * j)
#sub /= 5
sub.reset_index().to_csv('submission_mel256.csv', index=False)
sub

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
recording_id,,,,,,,,,,,,,,,,,,,,,
000316da7,6.131475,5.827643,5.797981,6.799577,5.796273,6.125611,5.990841,5.932399,5.761902,5.921197,...,5.766042,5.890722,5.782336,5.881665,6.112166,5.829025,5.833536,5.806896,5.828168,5.938757
003bc2cb2,5.724198,5.888658,5.750153,6.205756,5.741813,5.769280,5.772959,5.992037,5.756063,5.828450,...,5.715419,5.772920,7.414839,5.764676,5.730651,5.715467,5.763905,5.716420,5.704597,5.779227
0061c037e,5.866131,5.891880,5.812936,6.230371,5.980272,6.148660,5.788274,6.687434,5.733707,5.861344,...,5.781084,5.880323,5.788772,5.810925,5.756432,5.907004,6.363484,5.820596,5.867963,5.993826
010eb14d3,7.262718,5.811694,5.722626,5.859229,5.703892,5.800492,5.719412,5.769783,6.459881,5.737057,...,5.755183,5.749275,5.690035,5.683093,6.228316,5.725061,5.706801,5.785533,5.729039,5.806670
011318064,5.831588,5.923256,5.849032,6.367482,5.764895,5.995125,5.955082,6.045024,5.774883,5.784213,...,6.879610,6.502219,5.956153,5.854267,6.033731,5.855879,5.878801,5.946482,5.849731,5.846838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff68f3ac3,6.095378,5.754042,5.779375,5.983885,5.846912,6.809120,5.783919,6.048665,5.837245,5.747612,...,5.739567,6.735595,5.748512,5.802677,5.907592,5.894662,5.865224,5.789262,5.818602,6.716211
ff973e852,5.790874,5.793634,5.762015,6.022834,5.788300,6.095182,5.872490,6.761741,5.748581,6.104545,...,5.860186,6.400343,5.782334,6.307889,5.789532,5.852809,6.143707,5.743564,5.871077,5.973329
ffa5cf6d6,5.715497,5.858188,5.837330,6.631120,5.698455,5.967671,5.844965,6.181824,5.656303,6.000316,...,5.727733,6.714696,5.795084,6.025071,5.693321,5.807507,5.948531,5.655945,5.813834,5.893874


In [ ]:
sum([0.1, 0.233, 0.1, 0.233, 0.234])

In [ ]:
lrs = np.arange(5e-4,1.0e-3,5e-5).tolist()
lrs, len(lrs)

In [ ]:
print(torch.tensor([[1],[2],[3]]).size())
a*torch.tensor([[1],[2],[3]])